# Players Comparison

This notebook allows you to compare statistics between two NHL players.

In [6]:
#! /home/db/.pyenv/shims/python

import requests
import pandas as pd
from ipywidgets import widgets, VBox, HBox, interactive_output
from IPython.display import display, HTML
import json

# Base server URL to get nhl stats
BASE_URL = "http://localhost:8080"

# Initialize variables to store player data
player_a_data = None
player_b_data = None
player_a_id = None
player_b_id = None

## Player Search

In [16]:
#! /home/db/.pyenv/shims/python

player_a_name_search_pattern = "Connor McDavid"

player_b_name_search_pattern = "Doughty"

def search_players():
    global player_a_id, player_b_id, player_a_data, player_b_data
    
    try:
        if player_a_name_search_pattern:
            response_a = requests.get(f"{BASE_URL}/player", params={"q": player_a_name_search_pattern})
            if response_a.status_code == 200:
                result_a = response_a.json()
                if isinstance(result_a, list) and len(result_a) > 0:
                    player_a_id = result_a[0]['playerId']
                    print(f"✓ Found Player A: {result_a[0]['name']} (ID: {player_a_id})")
                else:
                    print(f"✗ Player A not found")
            else:
                print(f"✗ Error searching Player A: {response_a.status_code}")
        
        if player_b_name_search_pattern:
            response_b = requests.get(f"{BASE_URL}/player", params={"q": player_b_name_search_pattern})
            if response_b.status_code == 200:
                result_b = response_b.json()
                if isinstance(result_b, list) and len(result_b) > 0:
                    player_b_id = result_b[0]['playerId']
                    print(f"✓ Found Player B: {result_b[0]['name']} (ID: {player_b_id})")
                else:
                    print(f"✗ Player B not found")
            else:
                print(f"✗ Error searching Player B: {response_b.status_code}")
        
        # Fetch players data
        if player_a_id:
            response_a_stats = requests.get(f"{BASE_URL}/player/{player_a_id}/stats")
            if response_a_stats.status_code == 200:
                player_a_data = response_a_stats.json()
        
        if player_b_id:
            response_b_stats = requests.get(f"{BASE_URL}/player/{player_b_id}/stats")
            if response_b_stats.status_code == 200:
                player_b_data = response_b_stats.json()
    except Exception as e:
        print(f"Error: {str(e)}")
            
search_players()

✓ Found Player A: Connor McDavid (ID: 8478402)
✓ Found Player B: Drew Doughty (ID: 8474563)


## Season Selection

In [ ]:
# # Create dropdowns for season and game type selection
# player_a_season = widgets.Dropdown(
#     options=[],
#     description='Player A Season:',
#     style={'description_width': '150px'}
# )

# player_a_game_type_id = widgets.Dropdown(
#     options=[("Regular Season", 2), ("Playoffs", 3)],
#     value=2,
#     description='Player A Game Type:',
#     style={'description_width': '150px'}
# )

# player_b_season = widgets.Dropdown(
#     options=[],
#     description='Player B Season:',
#     style={'description_width': '150px'}
# )

# player_b_game_type_id = widgets.Dropdown(
#     options=[("Regular Season", 2), ("Playoffs", 3)],
#     value=2,
#     description='Player B Game Type:',
#     style={'description_width': '150px'}
# )

# def update_season_dropdowns():
#     # Update Player A seasons
#     if player_a_data and 'seasonTotals' in player_a_data:
#         seasons_a = sorted(list(set([s['season'] for s in player_a_data['seasonTotals'] if s['season']])))
#         player_a_season.options = [(str(s), s) for s in seasons_a]
#         if seasons_a:
#             player_a_season.value = seasons_a[-1]  # Select most recent season
    
#     # Update Player B seasons
#     if player_b_data and 'seasonTotals' in player_b_data:
#         seasons_b = sorted(list(set([s['season'] for s in player_b_data['seasonTotals'] if s['season']])))
#         player_b_season.options = [(str(s), s) for s in seasons_b]
#         if seasons_b:
#             player_b_season.value = seasons_b[-1]  # Select most recent season

# # This will be called after searching players
# # update_season_dropdowns()

# display(
#     VBox([
#         HBox([player_a_season, player_a_game_type_id]),
#         HBox([player_b_season, player_b_game_type_id])
#     ])
# )

## Season Totals Comparison

In [ ]:
# def display_season_totals():
#     if not player_a_data and not player_b_data:
#         print("Please search for players first.")
#         return
    
#     # Helper function to get season stats
#     def get_season_stats(player_data, season, game_type_id):
#         if not player_data or 'seasonTotals' not in player_data:
#             return None
        
#         for stat in player_data['seasonTotals']:
#             if stat['season'] == season and stat['gameTypeId'] == game_type_id:
#                 return stat
#         return None
    
#     # Get stats for both players
#     stats_a = get_season_stats(player_a_data, player_a_season.value, player_a_game_type_id.value)
#     stats_b = get_season_stats(player_b_data, player_b_season.value, player_b_game_type_id.value)
    
#     # Create comparison table
#     comparison_data = []
    
#     if stats_a or stats_b:
#         stat_fields = ['gamesPlayed', 'goals', 'assists', 'points', 'plusMinus', 'pim', 
#                       'powerPlayGoals', 'shorthandedGoals', 'gameWinningGoals', 'shots']
        
#         for field in stat_fields:
#             row = {'Stat': field}
#             if stats_a:
#                 row[f"Player A ({player_a_season.value})"] = stats_a.get(field, '-')
#             if stats_b:
#                 row[f"Player B ({player_b_season.value})"] = stats_b.get(field, '-')
#             comparison_data.append(row)
        
#         df_comparison = pd.DataFrame(comparison_data)
#         display(df_comparison.to_html(escape=False, index=False), HTML('<br>'))
#     else:
#         print("No season stats found for the selected criteria.")

# # Display button
# display_season_button = widgets.Button(
#     description='Display Season Totals',
#     button_style='success'
# )

# season_output = widgets.Output()

# def on_display_season_click(b):
#     season_output.clear_output()
#     with season_output:
#         display_season_totals()

# display_season_button.on_click(on_display_season_click)

# display(
#     VBox([
#         display_season_button,
#         season_output
#     ])
# )

## Aggregated Season Totals (All Seasons)

In [ ]:
# def display_aggregated_season_totals():
#     if not player_a_data and not player_b_data:
#         print("Please search for players first.")
#         return
    
#     # Helper function to aggregate stats
#     def aggregate_season_stats(player_data, game_type_id):
#         if not player_data or 'seasonTotals' not in player_data:
#             return None
        
#         stats = [s for s in player_data['seasonTotals'] if s['gameTypeId'] == game_type_id]
#         if not stats:
#             return None
        
#         aggregated = {
#             'gamesPlayed': sum(s.get('gamesPlayed', 0) for s in stats),
#             'goals': sum(s.get('goals', 0) for s in stats),
#             'assists': sum(s.get('assists', 0) for s in stats),
#             'points': sum(s.get('points', 0) for s in stats),
#             'pim': sum(s.get('pim', 0) for s in stats),
#             'powerPlayGoals': sum(s.get('powerPlayGoals', 0) for s in stats),
#             'shorthandedGoals': sum(s.get('shorthandedGoals', 0) for s in stats),
#             'gameWinningGoals': sum(s.get('gameWinningGoals', 0) for s in stats),
#             'shots': sum(s.get('shots', 0) for s in stats),
#         }
#         return aggregated
    
#     # Get aggregated stats for both players
#     agg_stats_a = aggregate_season_stats(player_a_data, player_a_game_type_id.value)
#     agg_stats_b = aggregate_season_stats(player_b_data, player_b_game_type_id.value)
    
#     # Create comparison table
#     comparison_data = []
    
#     if agg_stats_a or agg_stats_b:
#         stat_fields = ['gamesPlayed', 'goals', 'assists', 'points', 'pim', 
#                       'powerPlayGoals', 'shorthandedGoals', 'gameWinningGoals', 'shots']
        
#         for field in stat_fields:
#             row = {'Stat': field}
#             if agg_stats_a:
#                 row['Player A (All Seasons)'] = agg_stats_a.get(field, '-')
#             if agg_stats_b:
#                 row['Player B (All Seasons)'] = agg_stats_b.get(field, '-')
#             comparison_data.append(row)
        
#         df_comparison = pd.DataFrame(comparison_data)
#         display(df_comparison.to_html(escape=False, index=False), HTML('<br>'))
#     else:
#         print("No aggregated stats found.")

# # Display button
# display_agg_season_button = widgets.Button(
#     description='Display Aggregated Season Totals',
#     button_style='success'
# )

# agg_season_output = widgets.Output()

# def on_display_agg_season_click(b):
#     agg_season_output.clear_output()
#     with agg_season_output:
#         display_aggregated_season_totals()

# display_agg_season_button.on_click(on_display_agg_season_click)

# display(
#     VBox([
#         display_agg_season_button,
#         agg_season_output
#     ])
# )

## Career Totals Comparison

In [ ]:
# def display_career_totals():
#     if not player_a_data and not player_b_data:
#         print("Please search for players first.")
#         return
    
#     # Helper function to get career totals
#     def get_career_totals(player_data):
#         if not player_data or 'careerTotals' not in player_data:
#             return None
#         return player_data['careerTotals']
    
#     career_a = get_career_totals(player_a_data)
#     career_b = get_career_totals(player_b_data)
    
#     if not career_a and not career_b:
#         print("No career totals found.")
#         return
    
#     # Display Regular Season Career Totals
#     print("\n=== REGULAR SEASON CAREER TOTALS ===")
    
#     reg_season_data = []
#     if career_a and 'regularSeason' in career_a:
#         reg_a = career_a['regularSeason']
#         stat_fields = ['gamesPlayed', 'goals', 'assists', 'points', 'plusMinus', 'pim',
#                       'powerPlayGoals', 'shorthandedGoals', 'gameWinningGoals', 'shots', 'shootingPctg']
        
#         for field in stat_fields:
#             row = {'Stat': field}
#             row['Player A'] = reg_a.get(field, '-')
#             if career_b and 'regularSeason' in career_b:
#                 reg_b = career_b['regularSeason']
#                 row['Player B'] = reg_b.get(field, '-')
#             reg_season_data.append(row)
    
#     if reg_season_data:
#         df_reg = pd.DataFrame(reg_season_data)
#         display(df_reg.to_html(escape=False, index=False), HTML('<br>'))
    
#     # Display Playoffs Career Totals
#     print("\n=== PLAYOFFS CAREER TOTALS ===")
    
#     playoff_data = []
#     if career_a and 'playoffs' in career_a:
#         playoff_a = career_a['playoffs']
#         stat_fields = ['gamesPlayed', 'goals', 'assists', 'points', 'plusMinus', 'pim',
#                       'powerPlayGoals', 'shorthandedGoals', 'gameWinningGoals', 'shots', 'shootingPctg']
        
#         for field in stat_fields:
#             row = {'Stat': field}
#             row['Player A'] = playoff_a.get(field, '-')
#             if career_b and 'playoffs' in career_b:
#                 playoff_b = career_b['playoffs']
#                 row['Player B'] = playoff_b.get(field, '-')
#             playoff_data.append(row)
    
#     if playoff_data:
#         df_playoff = pd.DataFrame(playoff_data)
#         display(df_playoff.to_html(escape=False, index=False), HTML('<br>'))

# # Display button
# display_career_button = widgets.Button(
#     description='Display Career Totals',
#     button_style='success'
# )

# career_output = widgets.Output()

# def on_display_career_click(b):
#     career_output.clear_output()
#     with career_output:
#         display_career_totals()

# display_career_button.on_click(on_display_career_click)

# display(
#     VBox([
#         display_career_button,
#         career_output
#     ])
# )